In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from ExplicitModels import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [4]:
# activation = F.relu
# architecture = [784, 128, 64, 10]
# lambda_ = 0.99999
# epsilon = 0.2
# supervised_lambda_weight = 1e-3
# neural_lr_start = 0.001 
# neural_lr_stop = 0.0005 
# neural_lr_rule = "constant"
# neural_lr_decay_multiplier = 0.005
# neural_dynamic_iterations = 50
activation = F.relu
architecture = [784, 128, 64, 10]
lambda_ = 1
epsilon = 0.01
supervised_lambda_weight = 1e-3
neural_lr_start = 0.1 
neural_lr_stop = 0.0005 
neural_lr_rule = "constant"
neural_lr_decay_multiplier = 0.005
neural_dynamic_iterations = 1

model = CorInfoMaxBiDirectionalNudged(architecture, lambda_, epsilon, activation, use_stepLR = True, 
                                      sgd_nesterov = False, optimizer_type = "sgd", 
                                      optim_lr_ff = 1.0, optim_lr_fb = 1.0, stepLR_step_size = 10*3000,)

In [5]:
trn_acc_list = []
tst_acc_list = []
random_sign = False
n_epochs = 50

for epoch_ in range(n_epochs):
    Bcopy = torch.clone(model.B[0]["weight"])
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        x = x.to(device).view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        #y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        if random_sign:
            rnd_sgn = 2*np.random.randint(2) - 1
            supervised_lambda_weight = rnd_sgn * supervised_lambda_weight

        model.batch_step(  x, y_one_hot, supervised_lambda_weight,
                           neural_lr_start, neural_lr_stop, neural_lr_rule,
                           neural_lr_decay_multiplier, neural_dynamic_iterations,
                        )
        if idx % 1 == 0:
            trn_acc = evaluatePC(model, train_loader, device, False, 
                                 printing = False)
            tst_acc = evaluatePC(model, test_loader, device, False, 
                                 printing = False)
            trn_acc_list.append(trn_acc)
            tst_acc_list.append(tst_acc)

            print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))
            print("B_1 update difference : {}".format(torch.norm(model.B[0]['weight'] - Bcopy)))
        
        
    trn_acc = evaluatePC(model, train_loader, device, False, 
                         printing = False)
    tst_acc = evaluatePC(model, test_loader, device, False, 
                         printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))
    print("B_1 update difference : {}".format(torch.norm(model.B[0]['weight'] - Bcopy)))

1it [00:07,  7.86s/it]

Epoch : 1, Train Accuracy : 0.09751666666666667, Test Accuracy : 0.0974
B_1 update difference : 0.0


2it [00:15,  7.82s/it]

Epoch : 1, Train Accuracy : 0.09915, Test Accuracy : 0.1009
B_1 update difference : 0.0


3it [00:23,  7.82s/it]

Epoch : 1, Train Accuracy : 0.09871666666666666, Test Accuracy : 0.098
B_1 update difference : nan


3it [00:27,  9.19s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/kuacc/users/bbozkurt15/.conda/envs/bbozkurt15/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_46960/2798578348.py", line 22, in <module>
    trn_acc = evaluatePC(model, train_loader, device, False,
  File "../src/torch_utils.py", line 162, in evaluatePC
    for x, y in loader:
  File "/kuacc/users/bbozkurt15/.conda/envs/bbozkurt15/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 530, in __next__
    data = self._next_data()
  File "/kuacc/users/bbozkurt15/.conda/envs/bbozkurt15/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 570, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "/kuacc/users/bbozkurt15/.conda/envs/bbozkurt15/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possib

TypeError: object of type 'NoneType' has no len()

In [ ]:
model.Wff

In [ ]:
plt.imshow(torch2numpy(model.B[0]['weight']))

In [ ]:
torch2numpy(model.B[0]['weight'])

In [13]:
torch.ones(128,128).to('cuda') + 0.0*model.B[0]['weight']

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0')

In [22]:
# activation = F.relu
# architecture = [784, 128, 64, 10]
# lambda_ = 0.9999
# epsilon = 0.01
# supervised_lambda_weight = 1e-3
# neural_lr_start = 0.001 
# neural_lr_stop = 0.0005 
# neural_lr_rule = "constant"
# neural_lr_decay_multiplier = 0.005
# neural_dynamic_iterations = 50

# model = CorInfoMaxBiDirectionalNudged(architecture, lambda_, epsilon, activation, use_stepLR = True, 
#                                       sgd_nesterov = False, optimizer_type = "sgd", 
#                                       optim_lr_ff = 1, optim_lr_fb = 0.1, stepLR_step_size = 10*3000,)

activation = F.relu
architecture = [784, 128, 64, 10]
lambda_ = 0.99999
epsilon = 0.01
supervised_lambda_weight = 1e-3
neural_lr_start = 0.1 
neural_lr_stop = 0.0005 
neural_lr_rule = "constant"
neural_lr_decay_multiplier = 0.005
neural_dynamic_iterations = 50

model = CorInfoMaxBiDirectionalNudged(architecture, lambda_, epsilon, activation, use_stepLR = True, 
                                      sgd_nesterov = False, optimizer_type = "sgd", 
                                      optim_lr_ff = 1, optim_lr_fb = 0.5, stepLR_step_size = 10*3000,)

model = CorInfoMaxNudged(architecture, lambda_, epsilon, activation, use_stepLR = True, 
                         sgd_nesterov = False, optimizer_type = "sgd", 
                         optim_lr = 1, stepLR_step_size = 10*3000,)

x, y = next(iter(train_loader))
x, y = x.to(device), y.to(device)
x = x.to(device).view(x.size(0),-1).T
y_one_hot = F.one_hot(y, 10).to(device).T

neurons = model.fast_forward(x, no_grad = True)
# for jj in range(len(neurons)):
#     neurons[jj] = neurons[jj].requires_grad_()
    
# layers = [x] + neurons

# layers_copy = model.copy_neurons(layers)
neurons = model.fast_forward(x, no_grad = True)
neurons = model.run_neural_dynamics(x, y_one_hot, neurons, supervised_lambda_weight, 
                          neural_lr_start, neural_lr_stop, lr_rule = neural_lr_rule, 
                          lr_decay_multiplier = neural_lr_decay_multiplier, 
                          neural_dynamic_iterations = neural_dynamic_iterations)

# corinfo_cost = model.CorInfo_Cost(x, y, neurons).sum()
# corinfo_cost.backward()

# (model.Wff[0]['weight'].grad)
neurons

[tensor([[-0.1182, -0.2738, -0.1414,  ..., -0.0847,  0.0058, -0.4658],
         [-0.5830,  0.3235,  0.2114,  ...,  0.0414, -0.1767, -0.0274],
         [-0.0964, -0.1394, -1.0852,  ...,  0.0103, -0.4806, -0.4961],
         ...,
         [-0.7570,  0.2953, -0.1379,  ...,  0.2147,  0.2571, -0.3191],
         [-0.4147,  0.0109, -0.3434,  ..., -0.5843, -0.8091, -0.6236],
         [-0.3929, -0.1974, -0.4347,  ..., -0.1661,  0.0517, -0.5392]],
        device='cuda:0', requires_grad=True),
 tensor([[ 0.1010,  0.0707, -0.2729,  ...,  0.1449, -0.1843,  0.1068],
         [-0.8148, -0.5566, -0.2989,  ..., -0.2424,  0.4361, -0.2299],
         [-0.4215,  0.1320, -0.4752,  ..., -0.1623, -0.7825, -0.1080],
         ...,
         [-0.1272,  0.2122, -0.2905,  ..., -0.2768, -0.3176, -0.0832],
         [ 0.4737,  0.2753, -0.2721,  ..., -0.1342, -0.1076, -0.0320],
         [ 0.8081,  0.3926, -0.0470,  ..., -0.0111, -0.2150,  0.0753]],
        device='cuda:0', requires_grad=True),
 tensor([[ 3.2593e-02, -1.

In [23]:
torch.argmax(neurons[-1], 0), y

(tensor([6, 5, 0, 3, 0, 4, 0, 5, 6, 0, 6, 6, 0, 0, 5, 6, 6, 0, 0, 0],
        device='cuda:0'),
 tensor([6, 1, 7, 6, 8, 7, 3, 2, 1, 6, 9, 4, 5, 7, 1, 7, 0, 4, 7, 7],
        device='cuda:0'))

In [ ]:
plt.imshow(torch2numpy((model.Wff[0]['weight'].grad)))

In [ ]:
mbs = x.size(1)
for jj in range(len(neurons)):
    neurons[jj] = neurons[jj].requires_grad_()
corinfo_cost = model.CorInfo_Cost(x, y, neurons)
init_grads = torch.tensor([1 for i in range(mbs)], dtype=torch.float, device=device, requires_grad=True) #Initializing gradients
grads = torch.autograd.grad(corinfo_cost, neurons, grad_outputs=init_grads, create_graph=False) # dPhi/ds

In [ ]:
one_over_epsilon = model.one_over_epsilon
gam_ = model.gam_

x, y = next(iter(train_loader))
x, y = x.to(device), y.to(device)
x = x.to(device).view(x.size(0),-1).T
y_one_hot = F.one_hot(y, 10).to(device).T

Wff = model.Wff
B = model.B

neurons = model.fast_forward(x, no_grad = True)

layers = [x] + neurons
for jj in range(len(Wff)):
    if jj == 0:
        error = - one_over_epsilon * (layers[jj + 1] - (Wff[jj]['weight'] @ layers[jj] + Wff[jj]['bias'])) 
    else:
        error = - one_over_epsilon * (layers[jj + 1] - (Wff[jj]['weight'] @ model.activation(layers[jj]) + Wff[jj]['bias']))

    lateral_term = gam_ * 0.5 * (layers[jj + 1].T @ B[jj]['weight'] @ layers[jj + 1])
    corinfo_cost = torch.sum(error * error, 0)
    


In [ ]:
error.shape
torch.sum(error * error, 0).shape

In [ ]:
outer_prod_broadcasting((B[jj]['weight'] @ layers[jj + 1]), layers[jj + 1].T).shape

In [ ]:
layers[jj + 1][:,2].T @ B[jj]['weight'] @ layers[jj + 1][:,2]

In [ ]:
torch.sum((B[jj]['weight'] @ layers[jj + 1]) * layers[jj + 1], 0)

In [ ]:
(B[jj]['weight'] @ layers[jj + 1]).shape, layers[jj + 1].shape

In [ ]:
trn_acc_list = []
tst_acc_list = []
random_sign = False
n_epochs = 50

for epoch_ in range(n_epochs):
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        x = x.to(device).view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        #y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        if random_sign:
            rnd_sgn = 2*np.random.randint(2) - 1
            supervised_lambda_weight = rnd_sgn * supervised_lambda_weight

        model.batch_step(  x, y_one_hot, supervised_lambda_weight,
                           neural_lr_start, neural_lr_stop, neural_lr_rule,
                           neural_lr_decay_multiplier, neural_dynamic_iterations,
                        )

    trn_acc = evaluatePC(model, train_loader, device, False, 
                         printing = False)
    tst_acc = evaluatePC(model, test_loader, device, False, 
                         printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))